<table width="100%" border="3">
  <tbody>
    <tr>
      <td ><img src="https://aidea-web.tw/images/web/logo_white.png" alt="Aidea" width="400"/></td>
      <td align='left'><h1>AUAI Exercise 2: AOI Simple Pipeline </h1></td>
    </tr>
  </tbody>
</table>

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

https://drive.google.com/file/d/1Arcy_6pkMwLrg05A4f8S5dRzyu4I5j8Q/view?usp=sharing

In [ ]:
from google_drive_downloader import GoogleDriveDownloader
GoogleDriveDownloader.download_file_from_google_drive(file_id='1Arcy_6pkMwLrg05A4f8S5dRzyu4I5j8Q',dest_path='./content', unzip=True)

In [ ]:
import pandas as pd
df_train = pd.read_csv("train.csv",dtype=str)
print(df_train.shape)

In [ ]:
df_train.head()

In [ ]:
train_num = df_train.shape[0]
train_files = df_train.iloc[:train_num,0].values
train_labels = df_train.iloc[:train_num,1].values
print(train_labels[:20])

In [ ]:
import seaborn as sns
g = sns.countplot(train_labels)

In [ ]:
num_classes=6

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
model = InceptionResNetV2(include_top = True, input_shape=(299,299,3), weights=None, classes=num_classes)

In [ ]:
model.summary()

In [ ]:
#compile model using accuracy to measure model performance
#model.summary()
from tensorflow.keras import optimizers
model.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=3e-3),
            metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
img_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator = img_gen.flow_from_dataframe(dataframe=df_train,
            directory="train_images",
            x_col="ID",
            y_col="Label",
            subset=None,
            batch_size=8,
            shuffle=False,
            class_mode="categorical",
            color_mode="rgb",
            target_size=(299,299))

In [ ]:
train_generator.class_indices

In [ ]:
if train_generator.n % train_generator.batch_size ==0:
  step_size_train=train_generator.n//train_generator.batch_size
else:
  step_size_train=train_generator.n//train_generator.batch_size + 1


In [ ]:
print(train_generator.n)

In [ ]:
# Include the epoch in the file name (uses `str.format`)
import os
checkpoint_path = "training_cp/cp-{epoch:03d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights 
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True)

In [ ]:
# Create a callback that stop the model 
es_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [ ]:
hist = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, callbacks=[cp_callback, es_callback], epochs=100)

In [ ]:
hist = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, callbacks=[cp_callback, es_callback], epochs=100)

In [ ]:
hist = model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, callbacks=[cp_callback, es_callback], epochs=100)

In [ ]:
# Save the weights using the `checkpoint_path` format
model.load_weights("training_cp/cp-047.ckpt")


In [ ]:
model.save("AOI-inceptionv3-0625.h5")

In [ ]:
#y_predictions = model.predict(X_train, batch_size=20)
#train_generator.reset()
model.evaluate_generator(generator=train_generator, steps=step_size_train, verbose=1)

In [ ]:
#y_predictions = model.predict(X_train, batch_size=20)
train_generator.reset()
y_predictions = model.predict_generator(generator=train_generator, steps=step_size_train, verbose=1)

In [ ]:
print(y_predictions[:2])
type(y_predictions)

In [ ]:
y_predictions.shape

In [ ]:
predicts = np.argmax(y_predictions,axis=1)
print(predicts[0:10])

In [ ]:
type(predicts)

In [ ]:
df_pred = pd.DataFrame(y_predictions)

In [ ]:
df_pred.shape

In [ ]:
df_pred.to_csv('pred-iv3.csv')

In [ ]:
labels = train_labels.astype(int)
print(labels[:10])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion=confusion_matrix(labels, predicts)
print(confusion)

In [ ]:
overkill= []
underkill = []
for i in range(train_num):
  if labels[i] == 0 and predicts[i] !=0:
    overkill.append(i)
  if labels[i] != 0 and predicts[i] ==0:
    underkill.append(i)
print('# of overkill= {}; # of underkill= {} '.format(len(overkill), len(underkill)))

In [ ]:
for i in range(len(y_predictions)):
  if labels[i] != predicts[i]:
    print(i, ':', labels[i], '->', predicts[i], '=', y_predictions[i])

In [ ]:
test_generator = img_gen.flow_from_directory(
            directory="test_images",
            batch_size=8,
            shuffle=False,
            class_mode=None,
            color_mode="rgb",
            target_size=(299,299))

In [ ]:
df_test = pd.read_csv("test.csv",dtype=str)
print(df_test.shape)

In [ ]:
df_test.head()

In [ ]:
test_num = df_test.shape[0]
test_files  = df_test.iloc[:test_num,0].values
test_labels = df_test.iloc[:test_num,1].values
print(test_labels[:10])

In [ ]:
len(y_predictions)

In [ ]:
predicts = np.argmax(y_predictions,axis=1)
print(predicts[:20])

In [ ]:
img_gen2 = ImageDataGenerator(horizontal_flip=False, vertical_flip=False, preprocessing_function=preprocess_input)
test_generator = img_gen2.flow_from_dataframe(dataframe=df_test,
            directory="test_images",
            x_col="ID",
            y_col="Label",
            batch_size=32,
            shuffle=False,
            class_mode=None,
            target_size=(299,299))

In [ ]:
if test_generator.n % test_generator.batch_size ==0:
  step_size_test=test_generator.n//test_generator.batch_size
else:
  step_size_test=test_generator.n//test_generator.batch_size + 1

In [ ]:
#y_predictions = model.predict(X_train, batch_size=20)
test_generator.reset()
y_predictions = model.predict_generator(generator=test_generator, steps=step_size_test,verbose=1)

In [ ]:
import numpy as np
predicts=np.argmax(y_predictions,axis=1)

In [ ]:
len(predicts)

In [ ]:
df_out = pd.DataFrame(df_test)
df_out.shape

In [ ]:
df_out['Label'] = predicts
df_out.to_csv("0625-iv3.csv", index=False)

In [ ]:
from tensorflow.keras.applications import ResNet50V2
model = ResNet50V2(include_top = True, input_shape=(299,299,3), weights=None, classes=num_classes)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
model = MobileNetV2(include_top = True, input_shape=(299,299,3), weights=None, classes=num_classes)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import preprocess_input
img_gen = ImageDataGenerator(zca_whitening=False,preprocessing_function=preprocess_input)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
img_gen = ImageDataGenerator(zca_whitening=False,preprocessing_function=preprocess_input)